check out gephi.cql and behaviour_desktop.md

/media/lnr-ai/applications/./neo4j-desktop-offline-1.2.7-x86_64.AppImage

In [19]:
import pandas as pd
from py2neo import Graph
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
import time

#### Is the Neo4J db up and running?

In [20]:
graph.run("CALL db.schema.visualization()").data()

[{'nodes': [(_-4:Company {constraints: ['CONSTRAINT ON ( company:Company ) ASSERT (company.companyname) IS UNIQUE'], indexes: [], name: 'Company'}),
   (_-2:Merchant {constraints: ['CONSTRAINT ON ( merchant:Merchant ) ASSERT (merchant.franchisename) IS UNIQUE'], indexes: [], name: 'Merchant'}),
   (_-1:Client {constraints: ['CONSTRAINT ON ( client:Client ) ASSERT (client.dedupestatic) IS UNIQUE'], indexes: [], name: 'Client'}),
   (_-3:Segment {constraints: ['CONSTRAINT ON ( segment:Segment ) ASSERT (segment.seg_l3_num) IS UNIQUE'], indexes: [], name: 'Segment'})],
  'relationships': [(Client)-[:CLIENT_LINK_7916 {}]->(Client),
   (Client)-[:TRANSACTED_AT {}]->(Merchant),
   (Client)-[:CLIENT_LINK {}]->(Client),
   (Client)-[:COMPANY_TRANSACTED_AT {}]->(Company),
   (Company)-[:COMPANY_LINK {}]->(Company),
   (Merchant)-[:MERCHANT_VALUE_LINK {}]->(Merchant),
   (Merchant)-[:MERCHANT_LINK {}]->(Merchant),
   (Merchant)-[:TRIADIC_MERCHANT_FEET_LINK {}]->(Merchant),
   (Merchant)-[:MERCHAN

In [21]:
neo4j_query="""
MATCH (m:Merchant {subclass_id:56101})
WHERE EXISTS(m.triadic_56101_community) 
RETURN COUNT(m.triadic_56101_community) as membercount, m.triadic_56101_community as triadic_56101_community
ORDER BY membercount ASC;
"""


In [22]:
df=graph.run(neo4j_query).to_data_frame()

In [23]:
df

,membercount,triadic_56101_community
0,1,0
1,1,9724
2,1,1
3,1,9725
4,1,6
...,...,...
5127,351,5936
5128,364,6365
5129,398,8188
5130,517,1772


In [24]:
triadic_56101_community_list=list(df[df.membercount>20].triadic_56101_community)

In [25]:
len(triadic_56101_community_list)

24

In [31]:
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
hf=pd.DataFrame()
donelist=[]
leftoverlist=triadic_56101_community_list.copy()
counter=0
for community in triadic_56101_community_list:
    print('Doing community {community}, {counter} of {total}'.format(community=community,counter=len(donelist),total=len(triadic_56101_community_list)))
    graphname="""fastfood_{community}""".format(community=community)
    query="""CALL gds.graph.exists('{graphname}') YIELD exists;""".format(graphname=graphname)
    exists=graph.run(query).data()
    if exists[0]['exists']:
        print("graph exists!")
        graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
    #     CALL gds.graph.drop('my-store-graph') YIELD graphName;
    else:
        print("graph doesn't exist!")
    # ----------------------------------------------------------------------------
    create_graph_query="""
    CALL gds.graph.create.cypher(
        'fastfood_{community}',
        'MATCH (m:Merchant {{subclass_id:56101}}) WHERE EXISTS(m.triadic_56101_community) AND 
        m.triadic_56101_community = {community} RETURN ID(m) AS id',
        'MATCH (a:Merchant)-[:TRIADIC_MERCHANT_FEET_LINK]->(b:Merchant) WHERE ID(a)<ID(b) AND
        EXISTS(a.triadic_56101_community) AND EXISTS(b.triadic_56101_community) AND
        a.triadic_56101_community = {community} AND
        b.triadic_56101_community = {community}
         RETURN ID(a) AS source, ID(b) AS target'
    );
    """.format(community=community)
    graph.run(create_graph_query).data()

    pagerank_query="""CALL gds.pageRank.stream('{graphname}', {{ maxIterations: 20, dampingFactor: 0.85 }})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).franchisename AS franchisename, score
    ORDER BY score DESC, franchisename ASC;""".format(graphname=graphname)
    gf=graph.run(pagerank_query).to_data_frame()
    gf['community']=community
    gf['membercount']=list(df[df.triadic_56101_community==community]['membercount'])[0]
    gf.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_{community}'.format(community=community),index=False) 
    hf = hf.append(gf, ignore_index=True)
    print(gf)
    print("Delete graph {graphname}...".format(graphname=graphname))
    graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
#     print('-Sleep 60s---------------------------------')
#     time.sleep(60)
    donelist.append(community)
    leftoverlist.pop(leftoverlist.index(community))
    print(leftoverlist)
    counter+=1
    print("====================================================================================")

Doing community 7916, 0 of 24
graph doesn't exist!
      franchisename     score  community  membercount
0      KFC GROBLERS  0.809422       7916           25
1    NANDOS GROBLER  0.670787       7916           25
2      DEBONAIRS SI  0.660889       7916           25
3      KFC MOUTSIYA  0.601046       7916           25
4    WIMPY GROBLERS  0.575624       7916           25
5      KFC KWAGGAFO  0.470889       7916           25
6    STEERS MORATIW  0.441075       7916           25
7    DEBONAIRS GROB  0.394640       7916           25
8        KFC MOLOTO  0.277500       7916           25
9      McD Kwagga M  0.267938       7916           25
10     McD Groblers  0.259299       7916           25
11     KFC MOUTSE M  0.232283       7916           25
12   KFC MOUTSIYA M  0.213750       7916           25
13     KFC PHOLA PA  0.195549       7916           25
14   DEBONAIRS MORA  0.186734       7916           25
15   DEBONAIRS SIYA  0.186734       7916           25
16   DEBONAIRS JANE  0.150000  

             franchisename     score  community  membercount
0             WIMPY GOUDKO  3.199500       6314           55
1           WIMPY MOOI RIV  2.848526       6314           55
2             STEERS FLAMW  1.239143       6314           55
3             KFC MATLOLOS  1.139316       6314           55
4           DEBONAIRS POTC  0.924155       6314           55
5             KFC KLERKSDO  0.863793       6314           55
6           BURGER KING MO  0.646359       6314           55
7             McDONALDS PO  0.622306       6314           55
8             STEERS COMBO  0.603526       6314           55
9           Nandos Potchef  0.546116       6314           55
10          DEBONAIRS KLER  0.521069       6314           55
11          KFC POTCH GRIM  0.514512       6314           55
12          MCD Klerksdorp  0.496245       6314           55
13          KFC KLERKSDORP  0.479500       6314           55
14            KFC WILKOPPI  0.442130       6314           55
15            WIMPY MATL

         franchisename     score  community  membercount
0         STEERS PRELL  2.000298       6244          109
1         KFC Thaba 'N  1.531328       6244          109
2       MCD Bloemfonte  1.440592       6244          109
3       WIMPY LOCHLOGA  1.216754       6244          109
4       DEBONAIRS WELK  1.196247       6244          109
..                 ...       ...        ...          ...
104  STEERS CURIE PARK  0.150000       6244          109
105     WIMPY ALIWAL N  0.150000       6244          109
106       WIMPY TUBATS  0.150000       6244          109
107     WIMPY VIRGINIA  0.150000       6244          109
108         YUKON SPUR  0.150000       6244          109

[109 rows x 4 columns]
Delete graph fastfood_6244...
[7805, 7579, 1875, 7544, 5898, 4914, 154, 6778, 5936, 6365, 8188, 1772, 5989]
Doing community 7805, 11 of 24
graph doesn't exist!


TransientError: Database 'neo4j' unavailable

In [32]:
leftoverlist

[7805, 7579, 1875, 7544, 5898, 4914, 154, 6778, 5936, 6365, 8188, 1772, 5989]

In [33]:
# from py2neo import Graph
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
hf=pd.DataFrame()
donelist=[]
tmp_list=leftoverlist.copy()
# leftoverlist=tmp_list
counter=0
for community in leftoverlist:
    print('Doing community {community}, {counter} of {total}'.format(community=community,counter=len(donelist),total=len(tmp_list)))
    graphname="""fastfood_{community}""".format(community=community)
    query="""CALL gds.graph.exists('{graphname}') YIELD exists;""".format(graphname=graphname)
    exists=graph.run(query).data()
    if exists[0]['exists']:
        print("graph exists!")
        graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
    #     CALL gds.graph.drop('my-store-graph') YIELD graphName;
    else:
        print("graph doesn't exist!")
    # ----------------------------------------------------------------------------
    create_graph_query="""
    CALL gds.graph.create.cypher(
        'fastfood_{community}',
        'MATCH (m:Merchant {{subclass_id:56101}}) WHERE EXISTS(m.triadic_56101_community) AND 
        m.triadic_56101_community = {community} RETURN ID(m) AS id',
        'MATCH (a:Merchant)-[:TRIADIC_MERCHANT_FEET_LINK]->(b:Merchant) WHERE ID(a)<ID(b) AND
        EXISTS(a.triadic_56101_community) AND EXISTS(b.triadic_56101_community) AND
        a.triadic_56101_community = {community} AND
        b.triadic_56101_community = {community}
         RETURN ID(a) AS source, ID(b) AS target'
    );
    """.format(community=community)
    graph.run(create_graph_query).data()

    pagerank_query="""CALL gds.pageRank.stream('{graphname}', {{ maxIterations: 20, dampingFactor: 0.85 }})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).franchisename AS franchisename, score
    ORDER BY score DESC, franchisename ASC;""".format(graphname=graphname)
    gf=graph.run(pagerank_query).to_data_frame()
    gf['community']=community
    gf['membercount']=list(df[df.triadic_56101_community==community]['membercount'])[0]
    gf.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_{community}'.format(community=community),index=False) 
    hf = hf.append(gf, ignore_index=True)
    print(gf)
    print("Delete graph {graphname}...".format(graphname=graphname))
    graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
#     print('-Sleep 60s---------------------------------')
#     time.sleep(60)
    donelist.append(community)
    tmp_list.pop(tmp_list.index(community))
    print(tmp_list)
    counter+=1
    print("====================================================================================")

Doing community 7805, 0 of 13
graph doesn't exist!
          franchisename     score  community  membercount
0          MCDONALD'S C  2.147556       7805          115
1        SPUR EL SOMBRE  1.414796       7805          115
2          KFC BAKERS M  1.104362       7805          115
3          STEERS AUTOB  1.001734       7805          115
4        KFC NORTHWAY (  0.977861       7805          115
..                  ...       ...        ...          ...
110      WIMPY ESTCOURT  0.150000       7805          115
111      WIMPY GREYTOWN  0.150000       7805          115
112        WIMPY HOWICK  0.150000       7805          115
113  WIMPY LIBERTY MALL  0.150000       7805          115
114      WIMPY WOODBURN  0.150000       7805          115

[115 rows x 4 columns]
Delete graph fastfood_7805...
[7579, 1875, 7544, 5898, 4914, 154, 6778, 5936, 6365, 8188, 1772, 5989]
Doing community 7579, 1 of 12
graph doesn't exist!
          franchisename     score  community  membercount
0          KFC SUN

TransientError: Database 'neo4j' unavailable

In [37]:
# from py2neo import Graph
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
hf=pd.DataFrame()
donelist=[]
leftoverlist=tmp_list.copy()
# leftoverlist=tmp_list
counter=0
for community in leftoverlist:
    print('Doing community {community}, {counter} of {total}'.format(community=community,counter=len(donelist),total=len(tmp_list)))
    graphname="""fastfood_{community}""".format(community=community)
    query="""CALL gds.graph.exists('{graphname}') YIELD exists;""".format(graphname=graphname)
    exists=graph.run(query).data()
    if exists[0]['exists']:
        print("graph exists!")
        graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
    #     CALL gds.graph.drop('my-store-graph') YIELD graphName;
    else:
        print("graph doesn't exist!")
    # ----------------------------------------------------------------------------
    create_graph_query="""
    CALL gds.graph.create.cypher(
        'fastfood_{community}',
        'MATCH (m:Merchant {{subclass_id:56101}}) WHERE EXISTS(m.triadic_56101_community) AND 
        m.triadic_56101_community = {community} RETURN ID(m) AS id',
        'MATCH (a:Merchant)-[:TRIADIC_MERCHANT_FEET_LINK]->(b:Merchant) WHERE ID(a)<ID(b) AND
        EXISTS(a.triadic_56101_community) AND EXISTS(b.triadic_56101_community) AND
        a.triadic_56101_community = {community} AND
        b.triadic_56101_community = {community}
         RETURN ID(a) AS source, ID(b) AS target'
    );
    """.format(community=community)
    graph.run(create_graph_query).data()

    pagerank_query="""CALL gds.pageRank.stream('{graphname}', {{ maxIterations: 20, dampingFactor: 0.85 }})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).franchisename AS franchisename, score
    ORDER BY score DESC, franchisename ASC;""".format(graphname=graphname)
    gf=graph.run(pagerank_query).to_data_frame()
    gf['community']=community
    gf['membercount']=list(df[df.triadic_56101_community==community]['membercount'])[0]
    gf.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_{community}'.format(community=community),index=False) 
    hf = hf.append(gf, ignore_index=True)
    print(gf)
    print("Delete graph {graphname}...".format(graphname=graphname))
    graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
#     print('-Sleep 60s---------------------------------')
#     time.sleep(60)
    donelist.append(community)
    tmp_list.pop(tmp_list.index(community))
    print(tmp_list)
    counter+=1
    print("====================================================================================")

Doing community 5989, 0 of 1
graph doesn't exist!
            franchisename      score  community  membercount
0           MCD Vangate (  14.216367       5989          765
1           Nandos Grassy   3.626409       5989          765
2            MCDONALD'S W   2.843273       5989          765
3            KFC SANCTUAR   2.780964       5989          765
4            KFC WATERFRO   2.684825       5989          765
..                    ...        ...        ...          ...
760  WIMPY BLUE ROUTE MAL   0.150000       5989          765
761      WIMPY DEPARTURES   0.150000       5989          765
762        WIMPY HERMANUS   0.150000       5989          765
763  WIMPY TABLE BAY MALL   0.150000       5989          765
764     WIMPY TYGERVALLEI   0.150000       5989          765

[765 rows x 4 columns]
Delete graph fastfood_5989...
[]


In [ ]:
# graph.run("""CALL gds.graph.exists('fastfood_{community}') YIELD exists;""".format(community=community)).data

In [ ]:
graph.run("""CALL gds.graph.exists('fastfood_{community}');""".format(community=community)).data

In [12]:
community=5936
print('Doing community {community}'.format(community=community))
graphname="""fastfood_{community}""".format(community=community)
query="""CALL gds.graph.exists('{graphname}') YIELD exists;""".format(graphname=graphname)
exists=graph.run(query).data()
if exists[0]['exists']:
    print("graph exists!")
    graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
#     CALL gds.graph.drop('my-store-graph') YIELD graphName;
else:
    print("graph doesn't exist!")
# ----------------------------------------------------------------------------
create_graph_query="""
CALL gds.graph.create.cypher(
    'fastfood_{community}',
    'MATCH (m:Merchant {{subclass_id:56101}}) WHERE EXISTS(m.triadic_56101_community) AND 
    m.triadic_56101_community = {community} RETURN ID(m) AS id',
    'MATCH (a:Merchant)-[:TRIADIC_MERCHANT_FEET_LINK]->(b:Merchant) WHERE ID(a)<ID(b) AND
    EXISTS(a.triadic_56101_community) AND EXISTS(b.triadic_56101_community) AND
    a.triadic_56101_community = {community} AND
    b.triadic_56101_community = {community}
     RETURN ID(a) AS source, ID(b) AS target'
);
""".format(community=community)
graph.run(create_graph_query).data()

pagerank_query="""CALL gds.pageRank.stream('{graphname}', {{ maxIterations: 20, dampingFactor: 0.85 }})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).franchisename AS franchisename, score
ORDER BY score DESC, franchisename ASC;""".format(graphname=graphname)
gf=graph.run(pagerank_query).to_data_frame()
gf['community']=community
gf['membercount']=list(df[df.triadic_56101_community==community]['membercount'])[0]
hf = hf.append(gf, ignore_index=True)
print(gf)
print("Delete graph {graphname}...".format(graphname=graphname))
graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()

print("====================================================================================")

Doing community 5936
graph doesn't exist!
            franchisename     score  community  membercount
0            WIMPY NEW RE  3.887059       5936          351
1            KFC ELDORADO  2.729541       5936          351
2            WIMPY THE GL  2.362756       5936          351
3          MCD Protea Gle  2.208990       5936          351
4            STEERS MAPON  2.059187       5936          351
..                    ...       ...        ...          ...
346  STEERS NEWTOWN JUNCT  0.150000       5936          351
347        STEERS THE GLE  0.150000       5936          351
348      STEERS/DEBONAIRS  0.150000       5936          351
349          WICHITA SPUR  0.150000       5936          351
350         WIMPY NEWTOWN  0.150000       5936          351

[351 rows x 4 columns]
Delete graph fastfood_5936...


In [38]:
triadic_56101_community_list

[7916,
 2697,
 6246,
 6305,
 6314,
 290,
 6790,
 8577,
 2160,
 5959,
 6244,
 7805,
 7579,
 1875,
 7544,
 5898,
 4914,
 154,
 6778,
 5936,
 6365,
 8188,
 1772,
 5989]

### Delete community client link edges

Choose community:

In [42]:
community=2697

In [57]:
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
delete_query="""CALL apoc.periodic.iterate('MATCH (client0:Client)-[cl:CLIENT_LINK_{community}]-(client1:Client)
WHERE ID(client1)<ID(client0)
RETURN cl;'
, 'DELETE cl', {{batchSize:500, iterateList:true, parallel:true}})
yield batches, total return batches, total;""".format(community=community)
graph.run(delete_query).data()

[{'batches': 3, 'total': 1500}]

In [61]:
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
count_links_query="""MATCH (client0:Client)-[cl:CLIENT_LINK_{community}]-(client1:Client)
WHERE ID(client1)<ID(client0)
RETURN count(cl) as client_links;""".format(community=community)
# graph.run(count_links_query).data()
df=graph.run(count_links_query).to_data_frame()

In [62]:
df

,client_links
0,23747


In [60]:
# community=2697
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
client_link_query="""CALL apoc.periodic.iterate('MATCH (client0:Client)-[t0:TRANSACTED_AT]->(merchant:Merchant)<-[t1:TRANSACTED_AT]-(client1:Client)
WHERE ID(client0) < ID(client1) AND 
merchant.triadic_56101_community = {community} 
RETURN client0, client1, t0, t1;
', "MERGE (client0)-[link:CLIENT_LINK_{community}]-(client1)
ON CREATE SET link.count = 1,
link.ID0=ID(client0),
link.ID1=ID(client1),
link.transactioncount0 = t0.transactioncount,
link.transactioncount1 = t1.transactioncount,
link.transactionamount0 = t0.transactionamount,
link.transactionamount1 = t1.transactionamount
ON MATCH SET link.count = link.count + 1,
link.transactioncount0 = link.transactioncount0+t0.transactioncount,
link.transactioncount1 = link.transactioncount1+t1.transactioncount,
link.transactionamount0 = link.transactionamount0+t0.transactionamount,
link.transactionamount1 = link.transactionamount1+t1.transactionamount;", {{batchSize:500, iterateList:true, parallel:true}})
yield batches, total return batches, total;
""".format(community=community)
graph.run(client_link_query).data()

[{'batches': 747, 'total': 373261}]

### Check sub graph existence and delete

In [63]:
print('Doing community {community}'.format(community=community))
graphname="""client_link_graph_{community}""".format(community=community)
query="""CALL gds.graph.exists('{graphname}') YIELD exists;""".format(graphname=graphname)
exists=graph.run(query).data()
if exists[0]['exists']:
    print("graph exists!")
    graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
#     CALL gds.graph.drop('my-store-graph') YIELD graphName;
else:
    print("graph doesn't exist!")

Doing community 2697
graph doesn't exist!


### Create a subgraph for clients related via a client_link relationship between clients for ONE merchant cluster/community

In [66]:
subgraph_query="""CALL gds.graph.create.cypher(
    '{graphname}',
    'MATCH (c0:Client)-[:TRANSACTED_AT]->(m:Merchant) WHERE EXISTS(m.triadic_56101_community) AND 
    m.triadic_56101_community = {community} RETURN id(c0) AS id',
    'MATCH (c0:Client)-[rel:CLIENT_LINK_{community}]-(c1:Client) RETURN id(c0) AS source, id(c1) AS target, rel.count as count'
)
YIELD graphName, nodeCount, relationshipCount, createMillis;""".format(graphname=graphname,community=community)
graph.run(subgraph_query).data()

[{'graphName': 'client_link_graph_2697',
  'nodeCount': 3398,
  'relationshipCount': 47494,
  'createMillis': 3124}]

### Cluster this subgraph of clients and STREAM output

In [67]:
louvain_stream_query="""CALL gds.louvain.stream('client_link_graph_{community}', {{ relationshipWeightProperty: 'count', 
maxLevels:10, 
maxIterations:200, includeIntermediateCommunities:true}})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).dedupestatic AS dedupestatic, communityId, intermediateCommunityIds
ORDER BY communityId DESC;""".format(community=community)
# graph.run(louvain_stream_query).data()
df=graph.run(louvain_stream_query).to_data_frame()

In [ ]:
df

### Cluster this  subgraph of clients and WRITE output on each client node

In [ ]:
louvain_write_query="""CALL gds.louvain.write('client_link_graph_{community}', {{ relationshipWeightProperty: 'count', 
maxLevels:10, 
maxIterations:200, writeProperty: 'client_link_community_{community}' }})
YIELD communityCount, modularity, modularities""".format(community=community)
graph.run(louvain_write_query).data()

### Inspect client membership of merhcant community

In [ ]:
client_membership_query="""MATCH (c:Client) 
WHERE EXISTS(c.client_link_community_{community}) 
RETURN COUNT(c.client_link_community_{community}) as count, c.client_link_community_{community} as 
client_link_community
ORDER BY count ASC""".format(community=community)
# graph.run(louvain_stream_query).data()
df=graph.run(client_membership_query).to_data_frame()